In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import pandas as pd
import os
import requests
from datetime import datetime
from urllib.request import urlopen

In [ ]:
now = datetime.now()
today = now.strftime("%Y%m%d")
times = now.strftime("%H%m%s")

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/local/jdk8u222-b10'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/HDFS01/anaconda3/envs/main/bin/python'
conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.driver.memory', '2g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://nn01.bigdata:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("testreplace") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate();

In [ ]:
schema = StructType([
    StructField("job_id", StringType(), True),
    StructField("addjob_by", StringType(), True),
    StructField("customer", StringType(), True),
    StructField("site", StringType(), True),
    StructField("malltype", StringType(), True),
    StructField("worktype", StringType(), True),
    StructField("details", StringType(), True),
    StructField("onsite", StringType(), True),
    StructField("time", StringType(), True),
    StructField("travel", StringType(), True),
    StructField("engineer", StringType(), True),
    StructField("status", StringType(), True),
    StructField("day", StringType(), True),
    StructField("date_start", StringType(), True),
    StructField("date_end", StringType(), True),
    StructField("status_work", StringType(), True),
    StructField("end_job_detail", StringType(), True),
    StructField("file_location", StringType(), True),
    StructField("update_by", StringType(), True)
])

In [ ]:
df = spark.createDataFrame(pd.read_json('http://engineer.da.co.th/api_json.php'), schema=schema)
df = df.withColumn("details", regexp_replace("details", "\r\n", ""))

In [ ]:
df.write \
.mode('append') \
.option("sep","|") \
.csv(f"/user/hive/warehouse/pyspark.db/engineer_csv/{today}/{times}")

In [ ]:
os.system("sudo -u hdfs hdfs dfs -rmr /user/hive/warehouse/pyspark.db/engineer_csv")

In [ ]:
os.system("sudo -u hdfs hdfs dfs -ls /user/hive/warehouse/pyspark.db/engineer_csv/20230216/10021676517926")